<a href="https://colab.research.google.com/github/msdevanms/pyspark_training/blob/main/Data_Analysis_using_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Welcome**
[Here are the data that you need to import/mirror to your google drive, and then mount it](https://drive.google.com/drive/folders/1Crp5wbuowpRaliQbgBeSutVRapT7fGJv?usp=sharing)


### Let's mount the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Task 1 :
Installing pyspark module

In [ ]:
!pip install pyspark

Importing the modules

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
import matplotlib.pyplot as plts

creating spark session

In [ ]:
spark = SparkSession.builder.appName('spark_app').getOrCreate()

# Task 2 :
importing the *Listenings.csv* file:

In [ ]:
listening_csv_path = '/content/drive/My Drive/dataset/listenings.csv'
listening_df = spark.read.format('csv').option('inferSchema', True).option('header', True).load(listening_csv_path)

let's check the data:

In [ ]:
listening_df.show()

let's delete useless columns:

In [ ]:
listening_df = listening_df.drop('date')

drop the null rows:

In [ ]:
listning_df = listening_df.na.drop()

let's check the dataset again:

In [ ]:
listening_df.show()

let's see the schema:

In [ ]:
listening_df.printSchema()

let's see the shape of our dataframe:

In [ ]:
shape = (listening_df.count(), len(listening_df.columns))
print(shape)

# Task 3:

**Query #0:**
select two columns: track and artist

In [ ]:
q0 = listening_df.select('artist', 'track')
q0.show()

**Query #1**:

Let's find all of the records of those users who have listened to ***Rihanna***

In [ ]:
q1 = listening_df.select('*').filter(listening_df.artist == 'Rihanna')
q1.show()

**Query #2:**

Let's find top 10 users who are fan of ***Rihanna***

In [ ]:
q2 = listening_df.select('user_id').filter(listening_df.artist == 'Rihanna').groupby('user_id').agg(count('user_id').alias('count')).orderBy(desc('count')).limit(10)
q2.show()

**Query #3:**

find top 10 famous tracks

In [ ]:
q3 = listening_df.select('artist', 'track').groupby('artist', 'track').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q3.show()

**Query #4:**

find top 10 famous tracks of ***Rihanna***

In [ ]:
q4 = listening_df.select('artist', 'track').filter(listening_df.artist =='Rihanna').groupBy('artist', 'track').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q4.show()

**Query #5:**

find top 10 famous albums

In [ ]:
q5 = listening_df.select('artist', 'album').groupBy('artist', 'album').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q5.show()

# Task 4 :
importing the ***genre.csv*** file:

In [ ]:
genre_csv_path = '/content/drive/My Drive/dataset/genre.csv'
genre_df = spark.read.format('csv').option('inferSchema', True).option('header', True).load(genre_csv_path)

let's check the data

In [ ]:
genre_df.show()

Let's inner join these two data frames

In [ ]:
data = listening_df.join(genre_df, how = 'inner', on=['artist'])
data.show()

**Query #6**

find top 10 users who are fan of ***pop*** music

In [ ]:
q6 = data.select('user_id').filter(data.genre == 'pop').groupBy('user_id').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q6.show()

**Query #7**

find top 10 famous genres

In [ ]:
q7 = data.select('genre').groupBy('genre').agg(count('*').alias('count')).orderBy(desc('count')).limit(10)
q7.show()

# Task 5:
**Query #8**

find out each user favourite genre

In [ ]:
q8 = data.select('user_id', 'genre').groupBy('user_id', 'genre').agg(count('*').alias('count')).orderBy('user_id')
q8 .show()

**Query #9**

find out how many pop,rock,metal and hip hop singers we have

and then visulize it using bar chart

In [ ]:
q9 = genre_df.select('genre').filter( (col('genre')=='pop') | (col('genre')=='rock') | (col('genre')=='metal') | (col('genre')=='hip hop')).groupBy('genre').agg(count('genre').alias('count'))
q9.show()

Now, let's visualize the results using ***matplotlib***

In [ ]:
q9_list = q9.collect()

In [ ]:
labels = [row['genre'] for row in q9_list]
counts = [row['count'] for row in q9_list]

In [ ]:
print(labels)
print(counts)

now lets visualize these two lists using a bar chart

In [ ]:
plts.bar(labels,counts)